In [11]:
class singlePoint:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def get_x(self):
        return int(self.x)
    
    def get_y(self):
        return int(self.y)
    
    def position(self):
        return int(self.x),int(self.y)
    
class obstacle(singlePoint):
    def __init__(self,x,y,cost):
        self.x = x
        self.y = y
        self.cost = cost
        
    def get_cost(self):
        return int(self.cost)

class quickSand(obstacle):
    def __init__(self,x,y,cost):
        self.x = x
        self.y = y
        self.cost=cost
        
#work on the range
class spiderNet(obstacle):
    def __init__(self,x,y,cost):
        self.x = x
        self.y = y
        self.cost=cost
    
class fee(obstacle):
    def __init__(self,x,y,cost):
        self.x = x
        self.y = y
        self.cost = cost 


In [12]:
#read file and set up obstacles in arrays
import re

input_file = open("input2.txt","r")
lines = input_file.readlines()

line1 = lines[0]
line2 = lines[1]
line3 = lines[2]
line4 = lines[3]
line5 = lines[4]

input_file.close()

#CODE to get starting point in first line
starting_location_coordinates = re.findall('\d+', line1)
starting_coordinates = singlePoint(starting_location_coordinates[0],starting_location_coordinates[1])

#debugging
print ("starting location:" ,starting_coordinates.get_x(),",",starting_coordinates.get_y())


#CODE to get exit point in second line
exit_location_coordinates = re.findall('\d+', line2)
exit_coordinates = singlePoint(exit_location_coordinates[0],exit_location_coordinates[1])

#debugging
print ("exit location:" ,exit_coordinates.get_x(),",",exit_coordinates.get_y())

obstacles=[]

#CODE to get quickSand point in third line
quick_sands_location = re.findall('\d+', line3)
number_of_quick_sands = 0

for i in quick_sands_location[:]:
    number_of_quick_sands+=1

if (number_of_quick_sands % 2 != 0):
    number_of_quick_sands = number_of_quick_sands - 1


#quick_sand_array=[]
for i in range(0,(int)(number_of_quick_sands/2)):
    quick_sand = quickSand(quick_sands_location[i*2],quick_sands_location[i*2+1],10000)
    obstacles.append(quick_sand)
    
#CODE to get spider net point in fourth line
spider_net_location = re.findall('\d+', line4)
number_of_spider_nets=0

for i in spider_net_location[:]:
    number_of_spider_nets+=1

if ((number_of_spider_nets) % 2 != 0):
    number_of_spider_nets = number_of_spider_nets - 1

initialize_spider_net_array=[]
for i in range(0,int(number_of_spider_nets/2)):
        spider_net = spiderNet(spider_net_location[(i*2)],spider_net_location[(i*2+1)],0)
        initialize_spider_net_array.append(spider_net)

        
#total_spider_net=[]        
for spider in initialize_spider_net_array:
    x=int(spider.get_x())
    y=int(spider.get_y())
    for j in range(-1,2):
        #Avoid initializing spider web center twice
        if(j == 0):
            spider_net1 = spiderNet(x,y,10000)
            obstacles.append(spider_net1)
        else:
            spider_net1=spiderNet(x,y+j,10000)
            spider_net2=spiderNet(x+j,y,10000)
            obstacles.append(spider_net1)
            obstacles.append(spider_net2)
        
#debuging
#for web in total_spider_net:
    #print ("spider net at :",web.get_x() ,",",web.get_y() )      

#CODE to get fees point in fourth line
fees_location_and_cost = re.findall('\d+', line5)
number_of_fee_points = 0

for i in fees_location_and_cost[:]:
    number_of_fee_points+=1

#fees_array=[]
for i in range(0,(int)(number_of_fee_points/3)):
    fees = fee(fees_location_and_cost[i*3+1],fees_location_and_cost[i*3+2],fees_location_and_cost[i*3])
    obstacles.append(fees)
    
#debugging
#for fee in fees_array:
    #print ("fee at:", fee.get_x(),"," ,fee.get_y()," cost: ",fee.get_cost())

for obs in obstacles:
    if (isinstance(obs, quickSand)):
        print("Quicksand: {}, {}   {}".format(obs.x, obs.y, obs.cost))
    elif (isinstance(obs, spiderNet)):
        print("Spidernet: {}, {}   {}".format(obs.x, obs.y, obs.cost))
        
    elif (isinstance(obs, fee)):
        print("Fee: {}, {}   {}".format(obs.x, obs.y, obs.cost))
    

starting location: 2 , 3
exit location: 19 , 10
Quicksand: 0, 0   10000
Quicksand: 5, 5   10000
Quicksand: 15, 15   10000
Spidernet: 10, 9   10000
Spidernet: 9, 10   10000
Spidernet: 10, 10   10000
Spidernet: 10, 11   10000
Spidernet: 11, 10   10000
Spidernet: 17, 7   10000
Spidernet: 16, 8   10000
Spidernet: 17, 8   10000
Spidernet: 17, 9   10000
Spidernet: 18, 8   10000
Fee: 1, 3   2
Fee: 3, 3   10
Fee: 10, 15   7
Fee: 19, 9   200


In [13]:
class agent():
    current_x = 0
    current_y = 0
    def __init__ (self,starting_x,starting_y,exit_x,exit_y):
        self.starting_x=starting_x
        self.starting_y=starting_y
        self.exit_x=exit_x
        self.exit_y=exit_y
        
        #Current location
        self.current_x = starting_x
        self.current_y = starting_y
        
        #Calculates estimated cost to arrive at EXIT
        self.estimated_cost=abs(self.starting_x - self.exit_x)+abs(self.starting_y - self.exit_y)
        
        #Keeps track of cost
        self.actual_cost=0 
        
        
    #Assumed h() function value returned  
    def distance_to_exit(self):
        distance = abs(self.current_x - self.exit_x)
        distance += abs(self.current_y - self.exit_y)
        return distance
        

    def move_up(self):
        self.current_y +=1
        self.actual_cost+=1
        
    def move_down(self):
        self.current_y -=1
        self.actual_cost+=1
        
    def move_right(self):
        self.current_x +=1
        self.actual_cost+=1
        
    def move_left(self):
        self.current_x -=1
        self.actual_cost+=1
        
    def current_position(self):
        return (self.current_x,self.current_y)
    
    
    def look_up(self,step):
        temp_y = self.current_y
        temp_x = self.current_x
        temp_y += step
        
        #Attempts to match an obstacles coordinates with the one or two steps ahead
        for obs in obstacles:               
        #Returns cost of the obstacle and the obstacle's distance from the EXIT
            if ( temp_y == obs.get_y() and self.current_x == obs.get_x() ):
                return obs.get_cost() + abs(temp_x - self.exit_x)+ abs(temp_y - self.exit_y)
        #No obstacles matched, return indicated space (1 or 2) distance
        if temp_y > 19:
            return 100000000
        return abs(temp_x - self.exit_x)+ abs(temp_y - self.exit_y)
    
    def look_down(self,step):
        temp_y = self.current_y
        temp_x = self.current_x
        temp_y -= step
        for obs in obstacles:
            if ( temp_y == obs.get_y() and temp_x == obs.get_x()):
                return obs.get_cost() + abs(temp_x - self.exit_x)+ abs(temp_y - self.exit_y)
        if temp_y < 0:
            return 100000000
        return abs(temp_x - self.exit_x)+ abs(temp_y - self.exit_y)
        
    def look_right(self,step):
        temp_y = self.current_y
        temp_x = self.current_x
        temp_x += step
        for obs in obstacles:
            if ( temp_y == obs.get_y() and temp_x == obs.get_x()):
                return obs.get_cost() + abs(temp_x - self.exit_x)+ abs(temp_y - self.exit_y)
        if temp_x > 19:
            return 100000000
        return abs(temp_x - self.exit_x)+ abs(temp_y - self.exit_y)
        
    def look_left(self,step):
        temp_y = self.current_y
        temp_x = self.current_x
        temp_x -= step
        for obs in obstacles:
            if ( temp_y == obs.get_y() and temp_x == obs.get_x()):
                return obs.get_cost() + abs(temp_x - self.exit_x)+ abs(temp_y - self.exit_y)
        if temp_x < 0:
            return 100000000
        return abs(temp_x - self.exit_x)+ abs(temp_y - self.exit_y)
    
    def look_up_coordinates(self,step):
        return self.current_x , self.current_y+step
    
    def look_right_coordinates(self,step):
        return self.current_x+step , self.current_y
    def look_down_coordinates(self,step):
        return self.current_x , self.current_y - step
    def look_left_coordinates(self,step):
        return self.current_x , self.current_y - step



In [14]:
import queue 
q = queue.PriorityQueue() 
a=[]
path=[]

for i in range(0,4):
    a.append(0)

print (a)
#Insert the intial starting point to path list
double_o_seven = agent(starting_coordinates.get_x(),starting_coordinates.get_y(),exit_coordinates.get_x(),exit_coordinates.get_y())
#path.append(double_o_seven.current_position())
#print(path)
while( double_o_seven.distance_to_exit() >= 0 and double_o_seven.current_x >= 0 and  double_o_seven.current_x<20 
      and double_o_seven.current_y >= 0 and  double_o_seven.current_y<20):
    
    print("Current position:", double_o_seven.current_position(),
          " ", double_o_seven.distance_to_exit(), "<-- Distance")
    
    if(double_o_seven.distance_to_exit() == 0):
        print("Found EXIT\nMission Complete!\n")
        break
    else:
        #Heuristic values calculated from current position
        a[0]=double_o_seven.look_up(1)+double_o_seven.look_up(2)
        a[1]=double_o_seven.look_right(1)+double_o_seven.look_right(2)
        a[2]=double_o_seven.look_down(1)+double_o_seven.look_down(2)
        a[3]=double_o_seven.look_left(1)+double_o_seven.look_left(2)
        print(a)
        #print(a.index(min(a)))

        index_of_min_value = a.index(min(a))
        
        #Determine if agent's current postion was already visited
        for p in path[:-1]:
            if(double_o_seven.look_up_coordinates(1) == p) :
                 a[0] = 100000
            elif double_o_seven.look_right_coordinates(1) == p:
                a[1] = 100000
            elif double_o_seven.look_down_coordinates(1) == p:
                 a[2] = 100000
            elif double_o_seven.look_left_coordinates(1) == p:
                 a[3] = 100000
                    
        index_of_min_value = a.index(min(a))
        
        print("Minimum distance:", a[index_of_min_value], end = " ")
        if(double_o_seven.look_up_coordinates(1)== exit_coordinates.position()):
            print ("(Up)\n")
            index_of_min_value=0
        elif double_o_seven.look_right_coordinates(1)== exit_coordinates.position():
            print("(Right)\n")
            index_of_min_value=1
        elif double_o_seven.look_down_coordinates(1)== exit_coordinates.position():
            print("(Down)\n")
            print("(YAAAAAAAAAAAAAAAAAAAAAAAAAAAS)\n")
            index_of_min_value=2
        elif double_o_seven.look_left_coordinates(1)== exit_coordinates.position():
            print("(Left)\n")
            index_of_min_value=3
        
            
        #Choose the minimal value
        if(index_of_min_value == 0 ):
            print ("(Up)\n")
            double_o_seven.move_up()
        elif(index_of_min_value == 1 ):
            print("(Right)\n")
            double_o_seven.move_right()
        elif(index_of_min_value == 2 ):
            print("(Down)\n")
            double_o_seven.move_down()
        elif(index_of_min_value == 3 ):
            print("(Left)\n")
            double_o_seven.move_left()
        else:
            print("Array out of bounds ")

        path.append(double_o_seven.current_position())
    
#Print to file when EXIT found
#report(double_o_seven, path)
    
print(path)
print(double_o_seven.actual_cost)


[0, 0, 0, 0]
Current position: (2, 3)   24 <-- Distance
[45, 55, 51, 53]
Minimum distance: 45 (Up)

Current position: (2, 4)   23 <-- Distance
[43, 43, 49, 49]
Minimum distance: 43 (Up)

Current position: (2, 5)   22 <-- Distance
[41, 41, 47, 47]
Minimum distance: 41 (Up)

Current position: (2, 6)   21 <-- Distance
[39, 39, 45, 45]
Minimum distance: 39 (Up)

Current position: (2, 7)   20 <-- Distance
[37, 37, 43, 43]
Minimum distance: 37 (Up)

Current position: (2, 8)   19 <-- Distance
[35, 35, 41, 41]
Minimum distance: 35 (Up)

Current position: (2, 9)   18 <-- Distance
[35, 33, 39, 39]
Minimum distance: 33 (Right)

Current position: (3, 9)   17 <-- Distance
[33, 31, 37, 37]
Minimum distance: 31 (Right)

Current position: (4, 9)   16 <-- Distance
[31, 29, 35, 35]
Minimum distance: 29 (Right)

Current position: (5, 9)   15 <-- Distance
[29, 27, 33, 33]
Minimum distance: 27 (Right)

Current position: (6, 9)   14 <-- Distance
[27, 25, 31, 31]
Minimum distance: 25 (Right)

Current positio

In [15]:
import pylab as plot
plot.axis([0,20,0,20])

#GRID
#plot.grid(color='r', linestyle='-', linewidth=.1)
        #xstart,xend,ystart, yend
for i in range(1,20):
    plot.plot([i,i],[0,20],'b-',linewidth=.2)
    plot.plot([0,20],[i,i],'b-',linewidth=.2)

plot.plot(starting_coordinates.get_x(),starting_coordinates.get_y(),'go')
plot.plot(exit_coordinates.get_x(),exit_coordinates.get_y(),'go')
for ob in obstacles:
    plot.plot(ob.x,ob.y,"ro")
path_length=len(path)
for i in range(0,path_length):
    plot.plot(path[i][0], path[i][1],'ko')
plot.show()

In [9]:
#Reporting function
#Used after EXIT is found
def report(d007, path, step):
    output_file = open("output.txt","w")
    
    seq = ["Total cost: ", str(d007.actual_cost) + '\n',
           "Total number of steps taken: ", str(step) + '\n',
           "Traveled route: " ]
    
    output_file.writelines(seq)

    for coord in path[:]:
        if (coord == path[-1]): 
            output_file.write(str(coord)) 
        else:
            output_file.write(str(coord) + ', ')
        
    output_file.close()


In [83]:
#quick example of how the agent can move using manhattan distance not taking in consideration obstacles 
double_o_seven = agent(starting_coordinates.get_x(),starting_coordinates.get_y(),exit_coordinates.get_x(),exit_coordinates.get_y())
path=[]
print("initial point :",double_o_seven.starting_x,",",double_o_seven.starting_y)
print("final point :",double_o_seven.exit_x,",",double_o_seven.exit_y)
print("distance to exit:",double_o_seven.distance_to_exit())
print()
print("START!!:")
while( double_o_seven.distance_to_exit()>0
     and
     double_o_seven.current_x >= 0 and  double_o_seven.current_x<20
     and
     double_o_seven.current_y >= 0 and  double_o_seven.current_y<20):
    
        if(double_o_seven.current_x < exit_coordinates.get_x()):
            print("right")
            double_o_seven.move_right()
        elif(double_o_seven.current_y < exit_coordinates.get_y()): 
            print("up")
            double_o_seven.move_up()
            
        print("distance to exit:",double_o_seven.distance_to_exit())
        print("currently @:",double_o_seven.current_position() )
        print("exit @:",exit_coordinates.position() )
        path.append(double_o_seven.current_position())
        
print(path)

initial point : 2 , 3
final point : 19 , 10
distance to exit: 24

START!!:
right
distance to exit: 23
currently @: (3, 3)
exit @: ('19', '10')
right
distance to exit: 22
currently @: (4, 3)
exit @: ('19', '10')
right
distance to exit: 21
currently @: (5, 3)
exit @: ('19', '10')
right
distance to exit: 20
currently @: (6, 3)
exit @: ('19', '10')
right
distance to exit: 19
currently @: (7, 3)
exit @: ('19', '10')
right
distance to exit: 18
currently @: (8, 3)
exit @: ('19', '10')
right
distance to exit: 17
currently @: (9, 3)
exit @: ('19', '10')
right
distance to exit: 16
currently @: (10, 3)
exit @: ('19', '10')
right
distance to exit: 15
currently @: (11, 3)
exit @: ('19', '10')
right
distance to exit: 14
currently @: (12, 3)
exit @: ('19', '10')
right
distance to exit: 13
currently @: (13, 3)
exit @: ('19', '10')
right
distance to exit: 12
currently @: (14, 3)
exit @: ('19', '10')
right
distance to exit: 11
currently @: (15, 3)
exit @: ('19', '10')
right
distance to exit: 10
currentl